In [9]:
include("./tree.jl")
# import .tree
using JuMP, Gurobi, CSV,Random, DataFrames, StatsBase, Statistics

#
#cd("/Users/arkiratanglertsumpun/Documents/GitHub/parallel-node-search")
##------
#Read in the dataset
iris_full = CSV.read("iris.csv",DataFrame)
iris = iris_full[randperm(size(iris_full,1)),:][1:100,:]
x = Matrix(iris[:,1:4])
y = Vector(iris[:,5])
# #------
# lend_full = CSV.read("../lending-club/lend_training_70.csv")
# lend = lend_full[randperm(size(lend_full,1)),:][1:100,:]
# x = Matrix(select(lend,Not(:loan_status)))
# y = Vector(lend[:,:loan_status])
Y = tf.y_mat(y)

#----
# heart = CSV.read("heart.csv")
# # iris = iris_full[randperm(size(iris_full,1)),:]#[1:10,:]
# x = Matrix(heart[:,1:end-1])
# y = Vector(heart[:,end])

#----
n = size(x,1) # number of observations
p = size(x,2) # number of features
K = length(unique(y)) # number of labels k
N_min = 5
depth = 2
α = 0.0001

dt = fit(UnitRangeTransform, x, dims=1)
x = StatsBase.transform(dt,x)

#----
function _get_baseline(Y)
    Nt = sum((Y),dims=1)
    error = size(Y,1) - maximum(Nt)
    return error
end

function epsilon(x)
    ϵj = []
    n = size(x,1)
    for j in 1:p
        min_diff = 1
        diff = 0
        x_sorted = sort(x[:,j])
        for i in 1:n-1
            diff = abs(x_sorted[i+1] - x_sorted[i])
            if (diff < min_diff) & (diff!= 0)
                println(diff)
                min_diff = diff
            end
        end
        append!(ϵj,min_diff)
    end
    return(ϵj)
end

ϵ = epsilon(x)#.-1e-4
ϵmin = minimum(ϵ)
ϵmax = maximum(ϵ);


# optimize!(model)


0.027777777777777922
0.027777777777777915
0.02777777777777768
0.02777777777777765
0.04545454545454529
0.045454545454545275
0.04545454545454519
0.016949152542372895
0.01694915254237286
0.016949152542372836
0.04166666666666667
0.04166666666666666
0.04166666666666663
0.04166666666666652


In [10]:
tree = tf.get_OCT(2)
model = Model(optimizer_with_attributes(Gurobi.Optimizer,  "Threads" => 1))

#set_optimizer_attribute(model, "Presolve", 0)
# set_optimizer_attribute(model, "Cuts", 0)
#set_optimizer_attribute(model, "NoRelHeuristic", 0)
# model.set(GRB.IntParam.Presolve, 0);

Academic license - for non-commercial use only


In [11]:
# model = Model(Gurobi.Optimizer)
# model = Model(optimizer_with_attributes(Gurobi.Optimizer,  "Threads" => 1))
# model = Model(solver=GurobiSolver())
@variable(model,z[1:n,tree.leaves],Bin)
@variable(model,l[tree.leaves],Bin)
@variable(model,c[1:K,tree.leaves],Bin)
@variable(model,d[tree.branches],Bin) #
@variable(model,a[1:p,tree.branches],Bin)

@variable(model,Nt[tree.leaves]≥ 0)
@variable(model,Nkt[1:K,tree.leaves]≥ 0)
@variable(model,Lt[tree.leaves]≥ 0)
@variable(model,b[tree.branches]≥ 0)

# integer_relationship_constraints
@constraint(model,[t=tree.leaves],sum(c[:,t]) == l[t]) # OK
@constraint(model,[i=1:n,t=tree.leaves],z[i,t] ≤ l[t])
@constraint(model,[i=1:n],sum(z[i,:]) == 1)
@constraint(model,[t=tree.leaves],sum(z[:,t]) ≥ N_min*l[t])
@constraint(model,[t=tree.branches[2:end],p=tf.get_parent(t)],d[t] ≤ d[p])
@constraint(model,[t=tree.branches],sum(a[:,t]) == d[t])

# leaf_samples_constraints
@constraint(model,[t=tree.leaves], Nt[t] == sum(z[:,t])) #
@constraint(model,[k=1:K,t=tree.leaves],Nkt[k,t] == sum(z[i,t]*tf.y_mat(y)[i,k] for i=1:n))#(1 + tf.y_mat(y)[i,k])/2 for i=1:n))
# (1+Y)/2 for two class, K = 2
# leaf_error_constraints
@constraint(model,[t=tree.leaves,k=1:K],Lt[t] ≥ Nt[t] - Nkt[k,t] - n*(1-c[k,t]))
@constraint(model,[t=tree.leaves,k=1:K],Lt[t] ≤ Nt[t] - Nkt[k,t] + n*c[k,t])

# parent_branching_constraints
@constraint(model,[t=tree.branches],b[t]≤maximum(x)*d[t])

M = 1 # > maximum of x
@constraint(model,[i=1:n,t=tree.leaves,m=tf.R(t)],
            a[:,m]'*x[i,:] ≥ b[m]-(1-z[i,t])*M)
@constraint(model,[i=1:n,t=tree.leaves,m=tf.L(t)],
            a[:,m]'*(x[i,:].+ϵ.-ϵmin)+ϵmin ≤ b[m] + (1+ϵmax)*(1-z[i,t]))

L_baseline = _get_baseline(Y)
@objective(model, Min, sum(Lt)/L_baseline + α*sum(d))
optimize!(model)

Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 1356 rows, 454 columns and 7321 nonzeros
Model fingerprint: 0x4b024df3
Variable types: 23 continuous, 431 integer (431 binary)
Coefficient statistics:
  Matrix range     [1e-02, 1e+02]
  Objective range  [1e-04, 2e-02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+02]
Presolve time: 0.01s
Presolved: 1356 rows, 454 columns, 6537 nonzeros
Variable types: 3 continuous, 451 integer (431 binary)
Found heuristic solution: objective 1.0000000
Found heuristic solution: objective 0.8066516

Root relaxation: objective 0.000000e+00, 296 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0    3    0.80665    0.00000   100%     -    0s
H    0     0                       0.6292323    0.00000   100%     -    0s

In [ ]:

getobjectivevalue(model)

value.(Lt)
value.(a)
value.(b)
value.(Nt) # number of points in node t
value.(Nkt) # number of label k in node t
value.(d)
value.(c) # prediction label of each node
value.(l) # =1 if node leaf t contains any points
value.(z) # points in leaf node t


In [ ]:
value.(a)